## Cargando las librerías necesarias

In [22]:
# Limpio Cache
%load_ext autoreload
%autoreload 2

# Directorio raiz
import sys
sys.path.append("..")

from src.data.load_data import preprocess

from src import pd, RandomForestClassifier, f1_score, joblib

from src.features.features import features, target_col, ruta_guarda_modelo, EPS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Leemos los datos de enero 2020 (entrenamiento)

In [13]:
taxi = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-01.parquet')

## Definimos las características con las que realizaremos la clasificación.

Las construiremos a continuación en la etapa de pre-procesamiento de los datos.

In [14]:
taxi_train = preprocess(taxi, target_col, features, EPS)

## Ajustamos el modelo con los datos de entrenamiento

Este proceso se puede demorar bastante (¡en mi caso fueron 30 minutos!), pero podemos realizar una prueba mas pequeña reduciendo la cantidad de ejemplos de entrenamiento. Por ejemplo podríamos agregar la siguiente celda:

`taxi_train = taxi_train.head(100000)`

In [15]:
taxi_train = taxi_train.head(100000)

In [16]:
%%time

rfc = RandomForestClassifier(n_estimators=100, max_depth=10)

rfc.fit(taxi_train[features], taxi_train[target_col])

CPU times: total: 7.53 s
Wall time: 10.2 s


RandomForestClassifier(max_depth=10)

## Calculamos la métrica f1-score en el conjunto de entrenamiento

In [17]:
%%time
# Compute F1

preds = rfc.predict_proba(taxi_train[features])

CPU times: total: 484 ms
Wall time: 618 ms


In [18]:
preds_labels = [p[1] for p in preds.round()]

In [19]:
preds_labels[0:10]

[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [20]:
print(f'F1: {f1_score(taxi_train[target_col], preds_labels)}')

F1: 0.656204047803652


## Exportamos el modelo a un archivo

In [21]:
joblib.dump(rfc, ruta_guarda_modelo)

['../models/random_forest.joblib']